<a href="https://colab.research.google.com/github/Zack1243/Novel-Webscraper/blob/multi-threaded-downloads/Webscraper3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup as b4
import asyncio
from google.colab import files
import multiprocessing
import os
import requests
import time
import zipfile
!pip install selenium

In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [3]:
TRIAL = 3
NUM_THREADS = 4
LINKS = ["https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-1",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-2",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-3",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-4",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-5",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-6",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-7",
  "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities-1678/chapter-8"]

## Instantiate browser

In [4]:
def getBrowser():
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  browser = webdriver.Firefox(options=options)
  return browser

### Parse chapter

In [5]:
async def parseFile(html):

  # Start beautiful soup object and parse
  soup = b4(html, features="html.parser")
  tag = soup.article.find_all('span')

  # Get title
  for t in tag:
    if "Chapter" in t.get_text():
      title = t.get_text() + ".html"

  # Get chapter content
  soup = soup.find_all('p')
  with open(title, "w" , encoding='utf-8') as file:
    for paragraph in soup:
        file.write(str(paragraph) + "\n")  # Write each paragraph to the file
    print(str(soup))
  file.close()

### Download main webpage

In [6]:
async def getChapter(queue, browser, url, trial):
  # while True:
  try:
    browser.get(url)
    print(f"attempting to download url: {url}")
    timeout_in_seconds = 10
    WebDriverWait(browser, timeout_in_seconds).until(EC.presence_of_element_located((By.ID, 'chapter-container')))
    await parseFile(browser.page_source)
  except:
    if trial >= TRIAL:
      with open(url, "w") as file:
        file.write("I DONE GOOFED!!!")
      file.close()
      print("I give up...")
      queue.task_done()
    else:
      print(f"Download failed trial {trial}")
      trial += 1
      await getChapter(queue, browser, url, trial)

## Get Chapter page links

In [7]:
async def getChapterPage(queue, browser, url, trial):
  try:
    # Get webpage
    print(f"getChapterPages trying url: {url}...")
    browser.get(url)

    # Start beautiful soup html parsing
    soup = b4(browser.page_source, features="html.parser")
    links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")

    # Append chapter and chapter page links
    if links:
      queue.put_nowait("https://www.lightnovelpub.com" + links[0].find('a')['href'])
    strainer = soup.find(class_="chapter-list").find_all('a')
    for link in strainer:
      queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])


  # Try to get webpage a total of TRIAL times before giving up
  except TimeoutException:
    if trial >= TRIAL:
      with open(url, "w") as file:
        file.write("I DONE GOOFED!!!")
      file.close()
      print("I give up...")
    else:
      print(f"Download failed trial {trial}")
      await getChapterPage(queue, browser, url, trial + 1)

### Get links and go to next chapter page, if avaliable (testing from file)

In [8]:
async def getChapterPages(queue, browser):
  while True:

    # Fet item from queue
    url = await queue.get()

    print(f"Fetching url from queue: {url}")

    # If this is a chapter page
    if "page" in url:
      await getChapterPage(queue, browser, url, 0)

    # Else, this url is a chapter
    else:
      await getChapter(queue, browser, url, 0)
    print(f"Finished downloading/adding from url: {url}")
    # Mark as done
    queue.task_done()

## Get the first webpage

In [9]:
#TODO
  # Add the link to list of failed downloads if failed
def getFirstPage(url, browser, trial):
  try:
    queue = asyncio.Queue()

    # Get and parse webpage
    browser.get(url)
    soup = b4(browser.page_source, features="html.parser")

    # Get chapter links and chapter page links
    #TODO: Error check if the next page does not exist
    links = soup.find(class_="pagenav").find_all(class_="PagedList-skipToNext")
    strainer = soup.find(class_="chapter-list").find_all('a')

    # Found both chapter page links and chapter links
    if links:
      # Queue the next chapter page url
      url1 = "https://www.lightnovelpub.com" + links[0].find('a')['href']
      #print(f"Adding url to queue: {url1}")
      queue.put_nowait(url1)
      for link in strainer:
        #print(f"Adding url to queue: {link}")
        queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])
      return queue

    # Could only find chapter links
    elif strainer:
      for link in strainer:
        queue.put_nowait("https://www.lightnovelpub.com"+ link['href'])
      return queue

    # Got nothing
    else:
      return -1

  # Failed to download webpage
  except TimeoutException:
    # Give up!
    if trial >= TRIAL:
      # Write a file saying what failed
      print(f"Failed to find/get input url: {url}")
      return -1

    # Try again, up to 3 times
    else:
      print(f"Download failed trial {trial}")
      getFirstPage(url, browser, trial + 1)

## Async Download Chapter pages

In [10]:
async def asyncDownload(url, browser):
  NUM_THREADS = 4
  queue = getFirstPage(url, browser, 0)
  browser.quit()
  print(queue)
  if queue == -1:
    return

  # Adjust threads to queue size
  if queue.qsize() < NUM_THREADS:
    NUM_THREADS = queue.qsize()

  # Instantiate task list and browsers
  tasks = []
  browsers = []
  for i in range(NUM_THREADS):
    browsers.append(getBrowser())
    print(f"Starting browser {i}")
    task = asyncio.create_task(getChapterPages(queue, browsers[i]))
    tasks.append(task)

  # Wait for all queue items to be downloaded
  await queue.join()

  # Quit all browsers
  for browser in browsers:
    browser.quit()

  # Cancel all tasks
  for task in tasks:
    task.cancel()

  # Wait for tasks to be canceled
  await asyncio.gather(*tasks, return_exceptions=True)




  # Error checking
    # Check at the last chapter page
    # Check if there is a next chapter page
    # List of failed downloads

## Zip up files to be downloaded

In [11]:
def downloadZip(title):
  directory = os.getcwd()

  # List all files in the directory
  all_files = os.listdir(directory)

  # Specify the name of the zip file to create
  zip_file_name = title + ".zip"

  # Create a zip file
  with zipfile.ZipFile(zip_file_name, 'w') as zipf:
      # Add each file in the directory to the zip file
      for file in all_files:
          file_path = os.path.join(directory, file)
          zipf.write(file_path, os.path.basename(file_path))

  print("Zip file created:", zip_file_name)

  files.download(directory + '/' + zip_file_name)

## Make and change to directory (book's title)

In [12]:
def getDir(directory):
  if not os.path.exists(directory):
    os.mkdir(directory)
    os.chdir(directory)
  elif os.getcwd != directory:
    os.chdir(directory)
  else:
    pass

In [13]:
async def main1(url, title):
    # Change directory
    getDir(title)

    # Start async downloading chapter pages
    await asyncDownload(url, getBrowser())

    # Zip and download chapters
    downloadZip(title)

### Input url

In [14]:
# Ask user for ln information
#url = input("Enter URL: ")
#title = input("Enter directory: ")
url = "https://www.lightnovelpub.com/novel/advent-of-the-three-calamities/chapters"
title = "advant-of-the-three-calamities"
await main1(url, title)

Adding url to queue: https://www.lightnovelpub.com/novel/advent-of-the-three-calamities/chapters?page=2
Adding url to queue: <a href="/novel/advent-of-the-three-calamities-1678/chapter-1" title="Chapter 1: Prologue [1]">
<span class="chapter-no">1</span>
<strong class="chapter-title">
Prologue [1] </strong>
<time class="chapter-update" datetime="2024-05-09 12:40">
2 days ago
</time>
</a>
Adding url to queue: <a href="/novel/advent-of-the-three-calamities-1678/chapter-2" title="Chapter 2: Julien D. Evenus [1]">
<span class="chapter-no">2</span>
<strong class="chapter-title">
Julien D. Evenus [1] </strong>
<time class="chapter-update" datetime="2024-05-09 12:40">
2 days ago
</time>
</a>
Adding url to queue: <a href="/novel/advent-of-the-three-calamities-1678/chapter-3" title="Chapter 3: Julien D. Evenus [2]">
<span class="chapter-no">3</span>
<strong class="chapter-title">
Julien D. Evenus [2] </strong>
<time class="chapter-update" datetime="2024-05-09 12:40">
2 days ago
</time>
</a>
Add

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>